In [ ]:
import config
from phemex import *
import talib
import json
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import nest_asyncio
nest_asyncio.apply()
import asyncio
import websockets

# Create an instance of the Phemex exchange
exchange = initExchange()
symbol = config.TRADE_SYMBOL
trades = pd.DataFrame(columns=['timestamp','side','priceEp','volume'])
tradesPrices = []
ohlcvData = pd.DataFrame(columns=['open','high','low','close','volume','timestamp'])

In [ ]:
def getVwap(ohlcv):
    # Calculate the VWAP for each interval
    vwap = ohlcv.apply(lambda x: (x['close'] * x['volume']).sum() / x['volume'].sum())
    print('Bitcoin VWAP:', vwap)     
    # Calculate the weighted average VWAP for the 24-hour period
    weighted_vwap = (vwap * ohlcv['volume'].sum()) / vwap.sum()

    print('Bitcoin Weighted VWAP for the last 24 hours:', weighted_vwap)
    return {'vwap':vwap,'weighted_vwap':weighted_vwap}

In [ ]:
def createOrder(symbol,amount,side,type='market'):
    # Define the order parameters
    symbol = 'BTC/USD'  # The trading symbol
    amount = 1      # The amount of Bitcoin to buy
    side = 'buy'       # The side of the order
    type = 'market'    # The type of order

    # Place the order
    order = exchange.create_market_buy_order(symbol,amount)
    # Print the order details
    print(order)

In [ ]:
def create_or_update_stoploss(symbol,side,entry_price,stop_loss_price,amount):
    # Check if there is already an open stoploss order for this symbol and side
    existing_orders = exchange.fetch_open_orders(symbol=symbol)
    stoploss_order = None
    order_side = None
    # print(existing_orders) 
    difference = abs(entry_price - stop_loss_price)
    # print('side',side)
    if side == 'Buy':
        order_side = 'Sell'
        if  difference < 10:
            stop_loss_price - 10
    if side == 'Sell':
        order_side = 'Buy' 
        if  difference < 10:
            stop_loss_price + 10
    # print('order_side',order_side)
    # Filter out only stoploss orders
    for order in existing_orders:
        # print(order['info']['orderType'],order['info']['side'],order_side) 
        if order['info']['orderType'] == 'Stop' and order['info']['side'] == order_side:
            stoploss_order = order
            break

    if stoploss_order is None:
        # Create a new stoploss order
        print(f"Create a new stoploss order")
        # print(f'symbol={symbol},type=stop,side={order_side},amount={amount},price={entry_price},params=stopPx: {stop_loss_price},timeInForce: GoodTillCancel,  # Keep the order open until filled or cancelled')
        # order = exchange.create_order(
        #     symbol=symbol,
        #     type='stop',
        #     side=order_side,
        #     amount=amount,  # Amount of contracts, set to 0 for stop orders
        #     price=stop_loss_price
        #     # params={
        #     #     'stopPrice': stop_loss_price,  # Set the stop price
        #     #     'timeInForce': 'GoodTillCancel',  # Keep the order open until filled or cancelled
        #     # }
        # )
        
        order = exchange.create_order(
            symbol=symbol,
            type='stop',
            side=order_side,
            amount=amount,
            params={
                'stopPx': stop_loss_price,
                'ordType': 'Stop',
                'execInst': 'LastPrice', # if entry_price is None else 'MarkPrice',
                'px': entry_price,
            },
        )
        # print(f"Created new stoploss order: {order}")
        return order
    else:
        # Update the existing stoploss order
        print(f"Update the existing stoploss order")
        order = exchange.edit_order(
            id=stoploss_order['id'],
            symbol=symbol,
            amount=amount,
            params={
                'stopPx': stop_loss_price,
                'ordType': 'Stop',
                'execInst': 'LastPrice', # if entry_price is None else 'MarkPrice',
                'px': entry_price,
            },
        ) 
        # print(f"Updated existing stoploss order: {order}")
        return order

In [ ]:
def startStrategy(ticker,ohlcv):
    # Set the EMA periods
    ema_7 = 7
    ema_14 = 14
    ema_30 = 30
    posInfo = getPositionInfo()[0]
    # print('posInfo',posInfo)
    side = posInfo['side']
    unrealizedPerc = posInfo['unrealized_pnl_perc']
    amount = posInfo['amount']
    if side == 'None':
         signal = ohlcv.iloc[-1]['signal']
        #  print(f'signal: {signal}')
         if signal == 'Buy':
            # Place the Buy order
            order = exchange.create_market_buy_order(config.TRADE_SYMBOL_BTCUSD,amount)
            # Print the order details
            print(order)
         elif signal == 'Sell':
            # Place the Sell order
            order = exchange.create_market_sell_order(config.TRADE_SYMBOL_BTCUSD,amount)
            # Print the order details
            print(order)
    elif side == 'Buy' and unrealizedPerc > 20:
            # Place the Sell order
            order = exchange.create_market_sell_order(config.TRADE_SYMBOL_BTCUSD,amount)
            # Print the order details
            print(order)
    elif side == 'Sell' and unrealizedPerc < -20:
        # Place the Buy order
        order = exchange.create_market_buy_order(config.TRADE_SYMBOL_BTCUSD,amount)
        # Print the order details
        print(order)
    posInfo = getPositionInfo()[0]
    # print('posInfo',posInfo)         
    entryPrice = posInfo['entryPrice']
    
    
    # print(posInfo['amount'])
    # amount = posInfo['amount']
    symbol = posInfo['symbol']

    # get the current price for the symbol
    # ticker = exchange.fetch_ticker(symbol)
    # print(ticker)
    current_price = ticker['priceEp']*0.0001

    # Fetch historical candlestick data
    # ohlcv = exchange.fetch_ohlcv(symbol, config.TIMEFRAME, limit=1000)
    

    # Extract the close prices from the OHLCV data
    # print(ohlcv)
    close_prices = ohlcv['close']
    # print(close_prices)    
    # Convert close_prices to a numpy array
    close_prices = np.array(close_prices)
    # print(close_prices) 

    # Calculate the EMAs using talib
    ema_7_latest = (talib.EMA(close_prices, timeperiod=ema_7)[-1]*0.0001)
    ema_14_latest = (talib.EMA(close_prices, timeperiod=ema_14)[-1]*0.0001)
    ema_30_latest = (talib.EMA(close_prices, timeperiod=ema_30)[-1]*0.0001)
    # print('ema 7: {} ema 14: {} ema 30: {}'.format(ema_7_latest,ema_14_latest,ema_30_latest))
    # check if there are any open long positions
    has_long_positions = False

    if side == 'Buy':
        has_long_positions = True

    # print('# Check the long position conditions')
    if ema_7_latest > ema_14_latest and ema_30_latest > ema_7_latest and has_long_positions:
        print('# Place a buy order and set the stop loss slightly below the EMA 14')
        stop_loss_price = (ema_14_latest - 20)  # set the stop loss 10 USD below the EMA 14
        # print(f'# current_price {current_price} vs stoploss price {stop_loss_price}')
        order = create_or_update_stoploss(symbol, 'Buy',entryPrice,stop_loss_price,amount)
        # print(order)
        price = order['info']['price']
        # print('price: {} ema 7: {} ema 14: {} ema 30: {}'.format(price,ema_7_latest,ema_14_latest,ema_30_latest))

    elif ema_7_latest > ema_30_latest and ema_14_latest < ema_30_latest and has_long_positions:
        print('# Place a buy order and set the stop loss at the EMA 14')
        if current_price <= ema_14_latest:
            stop_loss_price = current_price - 20
        else:
            stop_loss_price = ema_14_latest  # set the stop loss at the EMA 14

        print(f'# current_price {current_price} vs stoploss price {stop_loss_price}')
        order = create_or_update_stoploss(symbol, 'Buy',entryPrice,stop_loss_price,amount)
        # print(order)
        price = order['info']['price']
        # print('price: {} ema 7: {} ema 14: {} ema 30: {}'.format(price,ema_7_latest,ema_14_latest,ema_30_latest))

    elif ema_7_latest > ema_30_latest and ema_14_latest > ema_30_latest and ema_14_latest < ema_7_latest and has_long_positions:
        print('# Place a buy order and set the stop loss slightly below the EMA 14')
        stop_loss_price = (ema_14_latest - 20) # set the stop loss 10 USD below the EMA 14
        # print(f'# current_price {current_price} vs stoploss price {stop_loss_price}')
        order = create_or_update_stoploss(symbol, 'Buy',entryPrice,stop_loss_price,amount)
        # print(order)
        price = order['info']['price']
        # print('price: {} ema 7: {} ema 14: {} ema 30: {}'.format(price,ema_7_latest,ema_14_latest,ema_30_latest))

    # print('# Check the short position conditions')
    elif ema_7_latest < ema_14_latest and ema_7_latest > ema_30_latest and ema_14_latest > ema_30_latest and not has_long_positions:
        print('# Place a sell order and set the stop loss slightly above the EMA 14')
        stop_loss_price = (ema_14_latest + 20) # set the stop loss 10 USD above the EMA 14
        # print(f'# current_price {current_price} vs stoploss price {stop_loss_price}')
        order = create_or_update_stoploss(symbol, 'Sell',entryPrice,stop_loss_price,amount)
        # print(order)
        price = order['info']['price']
        # print('price: {} ema 7: {} ema 14: {} ema 30: {}'.format(price,ema_7_latest,ema_14_latest,ema_30_latest))

    elif ema_7_latest < ema_30_latest and ema_14_latest > ema_30_latest and not has_long_positions:
        print('# Place a sell order and set the stop loss at the EMA 14')
        stop_loss_price = ema_14_latest # set the stop loss at the EMA 14
        # print(f'# current_price {current_price} vs stoploss price {stop_loss_price}')
        order = create_or_update_stoploss(symbol, 'Sell',entryPrice,stop_loss_price,amount)
        # print(order)
        price = order['info']['price']
        # print('price: {} ema 7: {} ema 14: {} ema 30: {}'.format(price,ema_7_latest,ema_14_latest,ema_30_latest))

    elif ema_14_latest < ema_30_latest and ema_7_latest < ema_14_latest and not has_long_positions:
        print('# Place a sell order and set the stop loss slightly above the EMA 14')
        stop_loss_price = (ema_14_latest + 20)# set the stop loss 10 USD above the EMA 14
        # print(f'# current_price {current_price} vs stoploss price {stop_loss_price}')
        order = create_or_update_stoploss(symbol, 'Sell',entryPrice,stop_loss_price,amount)
        # print(order)
        price = order['info']['price']
        print('price: {} ema 7: {} ema 14: {} ema 30: {}'.format(price,ema_7_latest,ema_14_latest,ema_30_latest))
    elif has_long_positions and current_price > ema_14_latest:
        print('# Place a buy order and set the stop loss slightly below the EMA 14')
        stop_loss_price = (ema_14_latest-10)# set the stop loss 10 USD above the EMA 14
        # print(f'# current_price {current_price} vs stoploss price {stop_loss_price}')
        order = create_or_update_stoploss(symbol, 'Buy',entryPrice,stop_loss_price,amount)
    elif not has_long_positions and current_price < ema_14_latest:
        print('# Place a buy order and set the stop loss slightly below the EMA 14')
        stop_loss_price = (ema_14_latest+10)# set the stop loss 10 USD above the EMA 14
        # print(f'# current_price {current_price} vs stoploss price {stop_loss_price}')
        order = create_or_update_stoploss(symbol, 'Sell',entryPrice,stop_loss_price,amount)
    else:
        # cancel all orders
        cancelResponse = exchange.cancel_all_orders(symbol,params={'untriggered':True})
        print('Remove stoploss: current_price: {} ema 7: {} ema 14: {} ema 30: {}'.format(current_price,ema_7_latest,ema_14_latest,ema_30_latest))


In [ ]:
def getTimeformat(interval):
    if 'S' in interval:
        timeformat = "%Y-%m-%d, %H:%M:%S"
    elif 'min' in interval:
        timeformat = "%Y-%m-%d, %H:%M"
    elif 'H' in interval:
        timeformat = "%Y-%m-%d, %H"    
    elif 'D' in interval:
        timeformat = "%Y-%m-%d"  
    elif 'M' in interval:
        timeformat = "%Y-%m"     
    elif 'Y' in interval:
        timeformat = "%Y"  
    else:
        timeformat = "%Y-%m-%d, %H:%M:%S"
    return timeformat

In [ ]:
async def getTradePrices(message,interval):
    # print('-- Function getTradePrices --')
    global trades
    try:        
        if len(trades) > 0:
            trades.append(json.loads(message)['trades'][0])
        else:
            trades = list(json.loads(message)['trades'])
        df = pd.DataFrame(trades,columns=['timestamp','side','priceEp','volume'])
        tick_datetime_object = pd.to_datetime(df['timestamp'], unit='ns',utc=True)
        timenow = pd.to_datetime(tick_datetime_object + timedelta(hours=1))
        df['timestamp'] = timenow
        df.set_index('timestamp',inplace=True)
        df.reset_index()
        tradesAgg = df.groupby([pd.Grouper(level='timestamp', freq=interval)]).agg({'side':lambda x: list(x),'priceEp':'max','volume':'sum'})
        
        return tradesAgg.dropna()
    except Exception as e:
            print('Foutmelding in functie getTradePrices: {}'.format(e))

In [ ]:
def addFibbLevels(ohlcv):
    # Calculate Fibonacci retracement levels
    high = ohlcv['high'].max()
    low = ohlcv['low'].min()
    diff = high - low
    ohlcv['fib_0.0'] = low
    ohlcv['fib_0.236'] = low + 0.236 * diff
    ohlcv['fib_0.5'] = low + 0.5 * diff
    ohlcv['fib_0.618'] = low + 0.618 * diff
    ohlcv['fib_0.65'] = low + 0.65 * diff
    ohlcv['fib_0.786'] = low + 0.786 * diff
    ohlcv['fib_1.0'] = high
    ohlcv['fib_1.618'] = high + 1.618 * diff
    
    # Add current price and signal
    current_price = ohlcv.iloc[-1]['close']
    ohlcv['signal'] = 'none'
    
    # Generate buy/sell signal
    if (current_price < ohlcv['fib_0.236'].iloc[-1]):
        ohlcv['signal'] = 'Buy'
    elif (current_price > ohlcv['fib_0.618'].iloc[-1] and current_price < ohlcv['fib_0.65'].iloc[-1]):
        ohlcv['signal'] = 'Sell'
    
    return ohlcv

In [ ]:
async def getOhlc(trades,interval='1min'):
    global ohlcvData
    try:   
        # print('-- Function getOhlc --')
        ohlc = pd.DataFrame(trades,columns=['open','high','low','close','volume','timestamp'])
        ohlc = trades['priceEp'].resample(interval).ohlc()
        tick_datetime_object = pd.to_datetime(ohlc.index, unit='ns',utc=True)
        timeformat = getTimeformat(interval)
        timenow = pd.to_datetime(pd.to_datetime(tick_datetime_object).strftime(timeformat))
        ohlc['timestamp'] = timenow.astype(np.int64) / 10**9    
        ohlc['volume'] = trades['volume'].resample(interval).mean() 
        # ohlc = await getVolumeColorDf(ohlc)
        ohlc = ohlc[ohlc['open'] > 0]
        ohlc.dropna()
        ohlcvData = pd.concat([ohlcvData,ohlc])
        ohlcvData = addFibbLevels(ohlcvData)
        return ohlcvData
    except Exception as e:
            print('Foutmelding in functie getOhlc: {}'.format(e))  

In [ ]:
async def send_message(message):
    try:
        global ohlcvData, tradesPrices
        if message != None:
            if 'trades' in message:
                tradesPrices = await getTradePrices(message,'1S')
        # print('Aantal trades: ',len(tradesPrices))
        if len(tradesPrices) > 100:
            ohlcvData = await getOhlc(tradesPrices,'1min')
            # print(ohlcvData)
            ticker = tradesPrices.iloc[-1]
            startStrategy(ticker,ohlcvData)
    except Exception as e:
        print('Error in send_message: {}'.format(e))

In [ ]:
# connect to Phemes websocket server
async def connectPhemexWS():
    global interval
    async for websocket in websockets.connect('wss://vapi.phemex.com/ws'):
        try:
            print('Phemex Websocket Server is connected!')
            subscribe_ticker_msg = json.dumps({
                "id": 0,
                "method": "tick.subscribe",
                "params": [config.MBTC_SYMBOL]
            })
            await websocket.send(subscribe_ticker_msg)
            
            subscribe_Trade_msg = json.dumps({
                "id": 0,
                "method": "trade.subscribe",
                "params": [config.TRADE_SYMBOL]
            })
            await websocket.send(subscribe_Trade_msg)  

            subscribe_orderbook_msg = json.dumps({
                "id": 0,
                "method": "orderbook.subscribe",
                "params": [config.TRADE_SYMBOL,True]
            })
            await websocket.send(subscribe_orderbook_msg)              
                     
            while True:
                message = await websocket.recv() 
                # if 'book' in message:
                #     print(message)          
                await send_message(message)
        except websockets.ConnectionClosed:
            continue

In [ ]:
# start python localhost websocket server
async def start_server():
    print('Server started')
    try:
        # await websockets.serve(register_new_client,"localhost",3000)
        await connectPhemexWS()
    except Exception as e:
        print('Error {}'.format(e))

In [ ]:
if __name__ == '__main__':
    event_loop = asyncio.get_event_loop()
    event_loop.run_until_complete(start_server())
    event_loop.run_forever()